In [1]:
#pip install pymongo
#pip install nltk

In [2]:
from pymongo import MongoClient
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import nltk
import re
#from tensorflow.keras import layers
#from tensorflow import keras
#import tensorflow as tf

from sklearn.model_selection import train_test_split

from ast import literal_eval

import matplotlib.pyplot as plt
import numpy as np

In [3]:
client = MongoClient("mongodb://localhost:27017/")  #name of the connection string 
db = client["DBLP"]             #name of the Database

#collection_dblp = db["citation_data"]    #Name of the item Name
collection_dblp = db["new Data"]
collection_arix = db["Arix"]  
# Fetch all data
cursor = collection_dblp.find()

#cursor1 = collection_arix.find()

In [4]:
df = pd.DataFrame(list(cursor))

In [5]:
#cursor = collection_dblp.find({}, {'year': 5})

In [6]:
#cursor1 = collection_dblp.find({'year': {'$gte': 1985}}, {'year': 1})

In [7]:
# data convert into the pandas dataframe
#df1 = pd.DataFrame(list(cursor1))


In [8]:
#df1.columns

In [9]:
# import matplotlib.pyplot as plt
# article_counts = df1['year'].value_counts().sort_index()

# # Plot the data
# plt.figure(figsize=(10, 6))
# article_counts.plot(kind='bar', color='skyblue')
# plt.title('Number of Articles Published Each Year')
# plt.xlabel('Year')
# plt.ylabel('Number of Articles')
# plt.xticks(rotation=90)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

In [10]:
#df.head()

In [11]:
#rearrange the columns with proper way
df=df[['_id','title','abstract','authors','n_citation','year','references','venue','id']]


In [12]:
#can be used to automatically convert object columns to a more specific data type
df = df.infer_objects()
df.dtypes

_id           object
title         object
abstract      object
authors       object
n_citation     int64
year           int64
references    object
venue         object
id            object
dtype: object

In [13]:
df.shape

(1000000, 9)

In [14]:
df_null=df.copy()

In [15]:

df_null.isnull().sum()
#.dropna(inplace=True)


_id                0
title              0
abstract      172467
authors            2
n_citation         0
year               0
references    124417
venue         177755
id                 0
dtype: int64

In [16]:
#Removing All Rows with NaN Values:
df_null.dropna(inplace=True)

In [17]:
df_null.isnull().sum()

_id           0
title         0
abstract      0
authors       0
n_citation    0
year          0
references    0
venue         0
id            0
dtype: int64

In [18]:
df_null.shape


(693622, 9)

In [19]:
df_null.drop_duplicates(subset=['_id'], keep='first', inplace=True)
df_null.reset_index(drop= True,inplace = True)

In [20]:
df_id=df_null[['id','title','abstract','n_citation','references']]
#df_referce=df_null[['id','references']]

In [21]:
#df_id.to_csv('chuck.csv')

In [22]:
df_id=df_id.head(100000)
# df_subset.to_csv('test.csv')# 


In [23]:
df_id1=df_id.copy()

In [24]:
df_id1.head(10)

,id,title,abstract,n_citation,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,"['09cb2d7d-47d1-4a85-bfe5-faa8221e644b', '10aa..."
1,4ab39729-af77-46f7-a662-16984fb9c1db,Attractor neural networks with activity-depend...,We studied an autoassociative neural network w...,50,"['4017c9d2-9845-4ad2-ad5b-ba65523727c5', 'b118..."
2,4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de,A characterization of balanced episturmian seq...,It is well-known that Sturmian sequences are t...,50,"['1c655ee2-067d-4bc4-b8cc-bc779e9a7f10', '2e4e..."
3,4ab3a98c-3620-47ec-b578-884ecf4a6206,Exploring the space of a human action,One of the fundamental challenges of recognizi...,221,"['056116c1-9e7a-4f9b-a918-44eb199e67d6', '05ac..."
4,4ab3b585-82b4-4207-91dd-b6bce7e27c4e,Generalized upper bounds on the minimum distan...,This paper generalizes previous optimal upper ...,0,"['01a765b8-0cb3-495c-996f-29c36756b435', '5dbc..."
5,4ab3e768-78c9-4497-8b8e-9e934cb5f2e4,Applying BCMP multi-class queueing networks fo...,Queueing networks with multiple classes of cus...,6,"['1c26e228-57d2-4b2c-b0c9-8d5851c17fac', '7539..."
6,4ab3f7cd-140b-4e29-99d4-f4e8006c4f65,A Push–Pull Class-C CMOS VCO,A CMOS oscillator employing differential trans...,50,"['0a09db01-264a-4bdf-942c-d33cceb35d3c', '36c9..."
7,4ab404e2-6f4b-4fb4-b093-50775e765b13,On computability of pattern recognition problems,In statistical setting of the pattern recognit...,2,['505f493b-e09d-444d-9ee2-5e5db6a5b8ac']
8,4ab4244d-fb3e-49a3-b125-367df3d8e6ba,Manipulating biological and mechanical micro-o...,We first discuss some general aspects of micro...,50,"['5ecd70e1-7ccc-4b2f-ac09-b91953cca5cd', '7fa7..."
9,4ab459ce-5f18-4cc0-9627-1dc557eb8b68,An abundance of invariant polynomials satisfyi...,"In 1999, Iwan Duursma defined the zeta functio...",0,"['02d38c7d-3baf-40e3-86d7-a8884ff503f4', '078e..."


In [25]:
#df_referce = df_referce.assign(references=df_referce['references'].str.split(',')).explode('references')

In [26]:
# import pandas as pd



# # Define a function to safely get the length of iterable objects
# def safe_len(value):
#     if isinstance(value, (list, str)):
#         return len(value)
#     return 0  # or return a default value, or handle as needed

# # Apply the function to calculate the count of references
# df_id1['ref_count'] = df_id1['references'].apply(safe_len)

# # Group by 'id', aggregate counts and collect titles
# result = df_id1.groupby('id').agg({
#     'ref_count': 'sum',  # Sum of references
#     'title': lambda titles: ', '.join(pd.Series(titles).dropna())  # Concatenate titles
# }).reset_index()

# # Sort by the total reference count in descending order
# result_sorted = result.sort_values(by='ref_count', ascending=False)



In [27]:
# df_id.head(2)

In [28]:

# Download NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')


## Preprocessing Steps for Natural Language Processing (NLP)

# Text Cleaning
### I. Converting to lowercase

In [29]:
df_id

,id,title,abstract,n_citation,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,"['09cb2d7d-47d1-4a85-bfe5-faa8221e644b', '10aa..."
1,4ab39729-af77-46f7-a662-16984fb9c1db,Attractor neural networks with activity-depend...,We studied an autoassociative neural network w...,50,"['4017c9d2-9845-4ad2-ad5b-ba65523727c5', 'b118..."
2,4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de,A characterization of balanced episturmian seq...,It is well-known that Sturmian sequences are t...,50,"['1c655ee2-067d-4bc4-b8cc-bc779e9a7f10', '2e4e..."
3,4ab3a98c-3620-47ec-b578-884ecf4a6206,Exploring the space of a human action,One of the fundamental challenges of recognizi...,221,"['056116c1-9e7a-4f9b-a918-44eb199e67d6', '05ac..."
4,4ab3b585-82b4-4207-91dd-b6bce7e27c4e,Generalized upper bounds on the minimum distan...,This paper generalizes previous optimal upper ...,0,"['01a765b8-0cb3-495c-996f-29c36756b435', '5dbc..."
...,...,...,...,...,...
99995,7116c792-5739-49a8-8150-3cf64124078a,Registration of 3D objects and surfaces,An approach to the problem of comparative anal...,91,"['00e6f131-0fce-4b6f-8500-c1d701bb3cf8', '0e59..."
99996,7117035a-0190-4404-bcec-3dc04404e005,Fetal electrocardiogram extraction by sequenti...,This work addresses the problem of fetal elect...,125,"['1fb87d62-3355-4868-96f6-e63ae59e677a', '4234..."
99997,7117606e-67f7-4c71-9c6c-2584da7e777c,The comment density of open source software code,The development processes of open source softw...,15,"['08c0d910-e673-4da8-bec8-ea8f6b8184fd', '2361..."
99998,711762b9-9799-4bc6-b890-e316094e070b,Supporting component and architectural re-usag...,We present an extended interface description l...,50,['1067759f-2494-49db-a9b2-a529d429a942']


In [30]:
df_id = df_id.applymap(lambda x: x.lower() if isinstance(x, str) else x)

### II. Removing URLs

In [31]:
# # Define a regex pattern to match URLs
# url_pattern = re.compile(r'https?://\S+')

# # Define a function to remove URLs from text
# def remove_urls(text):
#     return url_pattern.sub('', text)

# # Apply the function to the 'text' column and create a new column 'clean_text'
# df_id['title'] = df_id['title'].apply(remove_urls)
# df_id['abstract'] = df_id['abstract'].apply(remove_urls)

url_pattern = re.compile(r'https?://\S+')

# Define a function to remove URLs from text
def remove_urls(text):
    if isinstance(text, list):
        # Join list into a single string
        text = ' '.join(text)
    return url_pattern.sub('', text)

# Apply the function to the 'title' and 'abstract' columns and create new columns 'clean_title' and 'clean_abstract'
df_id['title'] = df_id['title'].apply(remove_urls)
df_id['abstract'] = df_id['abstract'].apply(remove_urls)



### III. Removing remove non-word and non-whitespace characters

In [32]:
df_id['title'] = df_id['title'].replace(to_replace=r'[^\w\s\b\d+]', value='', regex=True)
df_id['abstract'] = df_id['abstract'].replace(to_replace=r'[^\w\s\b\d+\b]', value='', regex=True)

In [33]:
# def clean_reference(reference_list):
#     # Convert list to string, then remove brackets from beginning, end, and middle
#     if isinstance(reference_list, list):
#         reference_str = ' '.join(reference_list)
#     else:
#         reference_str = reference_list
#     return re.sub(r'[^\w-]', '', reference_str)

# # Apply the cleaning function to the 'reference' column
# df_id['references'] = df_id['references'].apply(clean_reference)
def clean_reference(reference_list):
    # Check if input is a list
    if isinstance(reference_list, list):
        # Convert list to a single string
        reference_str = ', '.join(reference_list)
    else:
        reference_str = reference_list

    # Split the string by comma and clean each part
    parts = reference_str.split(', ')
    cleaned_parts = [re.sub(r'[^\w-]', '', part) for part in parts]

    # Recombine cleaned parts into a single string
    cleaned_reference_str = ', '.join(cleaned_parts)
    return cleaned_reference_str

# Apply the cleaning function to the 'reference' column
df_id['references'] = df_id['references'].apply(clean_reference)

In [34]:
df_id

,id,title,abstract,n_citation,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,a new approach of 3d watermarking based on ima...,in this paper a robust 3d triangular mesh wate...,50,"09cb2d7d-47d1-4a85-bfe5-faa8221e644b, 10aa16da..."
1,4ab39729-af77-46f7-a662-16984fb9c1db,attractor neural networks with activitydepende...,we studied an autoassociative neural network w...,50,"4017c9d2-9845-4ad2-ad5b-ba65523727c5, b1187381..."
2,4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de,a characterization of balanced episturmian seq...,it is wellknown that sturmian sequences are th...,50,"1c655ee2-067d-4bc4-b8cc-bc779e9a7f10, 2e4e57ca..."
3,4ab3a98c-3620-47ec-b578-884ecf4a6206,exploring the space of a human action,one of the fundamental challenges of recognizi...,221,"056116c1-9e7a-4f9b-a918-44eb199e67d6, 05ac52a1..."
4,4ab3b585-82b4-4207-91dd-b6bce7e27c4e,generalized upper bounds on the minimum distan...,this paper generalizes previous optimal upper ...,0,"01a765b8-0cb3-495c-996f-29c36756b435, 5dbc8ccb..."
...,...,...,...,...,...
99995,7116c792-5739-49a8-8150-3cf64124078a,registration of 3d objects and surfaces,an approach to the problem of comparative anal...,91,"00e6f131-0fce-4b6f-8500-c1d701bb3cf8, 0e592004..."
99996,7117035a-0190-4404-bcec-3dc04404e005,fetal electrocardiogram extraction by sequenti...,this work addresses the problem of fetal elect...,125,"1fb87d62-3355-4868-96f6-e63ae59e677a, 4234c39b..."
99997,7117606e-67f7-4c71-9c6c-2584da7e777c,the comment density of open source software code,the development processes of open source softw...,15,"08c0d910-e673-4da8-bec8-ea8f6b8184fd, 23612fe0..."
99998,711762b9-9799-4bc6-b890-e316094e070b,supporting component and architectural reusage...,we present an extended interface description l...,50,1067759f-2494-49db-a9b2-a529d429a942


In [35]:
df_referce = df_id1.assign(references=df_id1['references'].str.split(',')).explode('references')

In [36]:
df_referce

,id,title,abstract,n_citation,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,['09cb2d7d-47d1-4a85-bfe5-faa8221e644b'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,'10aa16da-3cc8-4af6-9d66-48037e915d76'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,'35cb45c3-9408-4096-ab30-bc2e4de3fb5d'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,'661a342e-a911-4420-b67d-51c75d3b14e9'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,A new approach of 3D watermarking based on ima...,"In this paper, a robust 3D triangular mesh wat...",50,'779553f3-e4c1-456e-bc01-5eb9d9567541'
...,...,...,...,...,...
99997,7117606e-67f7-4c71-9c6c-2584da7e777c,The comment density of open source software code,The development processes of open source softw...,15,'65a9df5a-0d41-498a-a436-48213c74ae06'
99997,7117606e-67f7-4c71-9c6c-2584da7e777c,The comment density of open source software code,The development processes of open source softw...,15,'e0828ce7-b08c-4026-866f-916b22d8844c']
99998,711762b9-9799-4bc6-b890-e316094e070b,Supporting component and architectural re-usag...,We present an extended interface description l...,50,['1067759f-2494-49db-a9b2-a529d429a942']
99999,71177a1f-48d1-4419-b529-b156a8de3af7,Optimal SIR algorithm vs. fully adapted auxili...,Particle filters (PF) and auxiliary particle f...,50,['18b0b2e7-9bc8-4b97-83e9-dce169ff2fbb'


In [37]:
df_refere=df_referce[['id','references']]
df_refere

,id,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,['09cb2d7d-47d1-4a85-bfe5-faa8221e644b'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,'10aa16da-3cc8-4af6-9d66-48037e915d76'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,'35cb45c3-9408-4096-ab30-bc2e4de3fb5d'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,'661a342e-a911-4420-b67d-51c75d3b14e9'
0,4ab3735c-80f1-472d-b953-fa0557fed28b,'779553f3-e4c1-456e-bc01-5eb9d9567541'
...,...,...
99997,7117606e-67f7-4c71-9c6c-2584da7e777c,'65a9df5a-0d41-498a-a436-48213c74ae06'
99997,7117606e-67f7-4c71-9c6c-2584da7e777c,'e0828ce7-b08c-4026-866f-916b22d8844c']
99998,711762b9-9799-4bc6-b890-e316094e070b,['1067759f-2494-49db-a9b2-a529d429a942']
99999,71177a1f-48d1-4419-b529-b156a8de3af7,['18b0b2e7-9bc8-4b97-83e9-dce169ff2fbb'


In [38]:
df_refere.to_csv('separte.csv')

### IV. Removing digits

In [39]:
df_id['title'] = df_id['title'].replace(to_replace=r'\d', value='', regex=True)
df_id['abstract'] = df_id['abstract'].replace(to_replace=r'\d', value='', regex=True)

## 2. Tokenization
#### Tokenization is the process of breaking down large blocks of text such as paragraphs and sentences into smaller, more manageable units.

In [40]:
df_id['title'] = df_id['title'].apply(word_tokenize)

In [41]:
df_id['abstract'] = df_id['abstract'].apply(word_tokenize)

In [42]:
df_id.head()

,id,title,abstract,n_citation,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,"[a, new, approach, of, d, watermarking, based,...","[in, this, paper, a, robust, d, triangular, me...",50,"09cb2d7d-47d1-4a85-bfe5-faa8221e644b, 10aa16da..."
1,4ab39729-af77-46f7-a662-16984fb9c1db,"[attractor, neural, networks, with, activityde...","[we, studied, an, autoassociative, neural, net...",50,"4017c9d2-9845-4ad2-ad5b-ba65523727c5, b1187381..."
2,4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de,"[a, characterization, of, balanced, episturmia...","[it, is, wellknown, that, sturmian, sequences,...",50,"1c655ee2-067d-4bc4-b8cc-bc779e9a7f10, 2e4e57ca..."
3,4ab3a98c-3620-47ec-b578-884ecf4a6206,"[exploring, the, space, of, a, human, action]","[one, of, the, fundamental, challenges, of, re...",221,"056116c1-9e7a-4f9b-a918-44eb199e67d6, 05ac52a1..."
4,4ab3b585-82b4-4207-91dd-b6bce7e27c4e,"[generalized, upper, bounds, on, the, minimum,...","[this, paper, generalizes, previous, optimal, ...",0,"01a765b8-0cb3-495c-996f-29c36756b435, 5dbc8ccb..."


## 3. Stopword Removal
#### Stopwords refer to the most commonly occurring words in any natural language.

In [43]:
stop_words = set(stopwords.words('english'))
df_id['title'] = df_id['title'].apply(lambda x: [word for word in x if word not in stop_words])
df_id['abstract'] = df_id['abstract'].apply(lambda x: [word for word in x if word not in stop_words])

## 4. Stemming/Lemmatization

In [44]:
# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Define a function to perform stemming on the 'text' column
def stem_words(words):
    return [stemmer.stem(word) for word in words]

In [45]:
# Apply the function to the 'text' column and create a new column 'stemmed_text'
df_id['title'] = df_id['title'].apply(stem_words)
df_id['abstract'] = df_id['abstract'].apply(stem_words)

In [46]:
df_top=df_id.head()

In [47]:
# #nltk.download('all')
# #nltk.download('averaged_perceptron_tagger')
# # initialize lemmatizer
# lemmatizer = WordNetLemmatizer()

# # define function to lemmatize tokens
# def lemmatize_tokens(tokens):
#     # convert POS tag to WordNet format
#     def get_wordnet_pos(word):
#         tag = nltk.pos_tag([word])[0][1][0].upper()
#         tag_dict = {"J": wordnet.ADJ,
#                     "N": wordnet.NOUN,
#                     "V": wordnet.VERB,
#                     "R": wordnet.ADV}
#         return tag_dict.get(tag, wordnet.NOUN)
    
#     # lemmatize tokens
#     lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    
#     # return lemmatized tokens as a list
#     return lemmas

#     df_id['lemmatized_title'] = df_id['title'].apply(lemmatize_tokens)

In [48]:
# # apply lemmatization function to column of dataframe
# df_id['lemmatized_title'] = df_id['title'].apply(lemmatize_tokens)

In [49]:
# df_id['lemmatized_abstract'] = df_id['abstract'].apply(lemmatize_tokens)

In [50]:
# import base64

# # Assuming df_id is your DataFrame containing the ObjectId values in the column '_id'
# df_id1 = df_id['_id']

# # Convert ObjectId values to strings
# df_id1_str = df_id1.astype(str)

# # Apply base64 encoding to each element of the Series
# encoded_values = df_id1_str.apply(lambda x: base64.b64encode(x.encode()))

# # Print the encoded values
# print("Encoded values:")
# print(encoded_values)



In [51]:
# import hashlib

# id_str = "4ab3735c-80f1-472d-b953-fa0557fed28b"
# hashed_value = hashlib.sha256(id_str.encode()).hexdigest()
# # print("Hashed value:", hashed_value)


In [52]:
df_id.head()

,id,title,abstract,n_citation,references
0,4ab3735c-80f1-472d-b953-fa0557fed28b,"[new, approach, watermark, base, imag, segment]","[paper, robust, triangular, mesh, watermark, a...",50,"09cb2d7d-47d1-4a85-bfe5-faa8221e644b, 10aa16da..."
1,4ab39729-af77-46f7-a662-16984fb9c1db,"[attractor, neural, network, activitydepend, s...","[studi, autoassoci, neural, network, dynam, sy...",50,"4017c9d2-9845-4ad2-ad5b-ba65523727c5, b1187381..."
2,4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de,"[character, balanc, episturmian, sequenc]","[wellknown, sturmian, sequenc, non, ultim, per...",50,"1c655ee2-067d-4bc4-b8cc-bc779e9a7f10, 2e4e57ca..."
3,4ab3a98c-3620-47ec-b578-884ecf4a6206,"[explor, space, human, action]","[one, fundament, challeng, recogn, action, acc...",221,"056116c1-9e7a-4f9b-a918-44eb199e67d6, 05ac52a1..."
4,4ab3b585-82b4-4207-91dd-b6bce7e27c4e,"[gener, upper, bound, minimum, distanc, psk, b...","[paper, gener, previou, optim, upper, bound, m...",0,"01a765b8-0cb3-495c-996f-29c36756b435, 5dbc8ccb..."


# 1. Loading and Preprocessing Data

In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import chromadb
from chromadb.config import Settings

# Ensure the 'title' and 'abstract' columns contain strings
df_id['title'] = df_id['title'].astype(str)
df_id['abstract'] = df_id['abstract'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

# Concatenate title and abstract
df_id['combined_text'] = df_id['title'] + " " + df_id['abstract']

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=768)  # Adjust the max_features based on your needs

# Fit and transform the combined text column
df_id['combined_embeddings'] = list(vectorizer.fit_transform(df_id['combined_text']).toarray())

# Ensure IDs are strings
df_id['id'] = df_id['id'].astype(str)



In [54]:
# Connect to Chroma and create a collection
client = chromadb.Client(Settings())
combined_collection = client.create_collection("combined_embeddings")

# Convert numpy arrays to lists
combined_embeddings = [emb.tolist() for emb in df_id["combined_embeddings"]]

# Split data into smaller batches
def split_into_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Add embeddings to the collection in batches
batch_size = 5461  # Set to the maximum allowed batch size
ids_batches = list(split_into_batches(df_id["id"].tolist(), batch_size))
combined_embeddings_batches = list(split_into_batches(combined_embeddings, batch_size))
combined_metadatas_batches = list(split_into_batches(df_id[["title", "abstract"]].to_dict(orient="records"), batch_size))

for ids_batch, combined_embeddings_batch, combined_metadatas_batch in zip(ids_batches, combined_embeddings_batches, combined_metadatas_batches):
    combined_collection.add(ids=ids_batch, embeddings=combined_embeddings_batch, metadatas=combined_metadatas_batch)


In [55]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.spatial.distance import cosine
# import chromadb
# from chromadb.config import Settings


# # Ensure the 'title' and 'abstract' columns contain strings
# df_id['title'] = df_id['title'].astype(str)
# df_id['abstract'] = df_id['abstract'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

# # Example additional metadata for re-ranking
# df_id['citation_count'] = df_id['n_citation']  # Use actual citation counts

# # Create TF-IDF vectorizer
# vectorizer = TfidfVectorizer(max_features=768)  # Adjust the max_features based on your needs

# # Fit and transform the title and abstract columns
# df_id['title_embeddings'] = list(vectorizer.fit_transform(df_id['title']).toarray())
# df_id['abstract_embeddings'] = list(vectorizer.fit_transform(df_id['abstract']).toarray())

# # Convert lists in 'abstract' to strings for metadata
# df_id['abstract_str'] = df_id['abstract']

# # Ensure IDs are strings
# df_id['_id'] = df_id['_id'].astype(str)


# 2. Embedding Storage in ChromaDB

In [56]:
# # Connect to Chroma and create collections
# client = chromadb.Client(Settings())
# title_collection = client.create_collection("title_embeddings")
# abstract_collection = client.create_collection("abstract_embeddings")

# # Convert numpy arrays to lists
# title_embeddings = [emb.tolist() for emb in df_id["title_embeddings"]]
# abstract_embeddings = [emb.tolist() for emb in df_id["abstract_embeddings"]]

# # Split data into smaller batches
# def split_into_batches(data, batch_size):
#     for i in range(0, len(data), batch_size):
#         yield data[i:i + batch_size]

# # Add embeddings to collections in batches
# batch_size = 5461  # Set to the maximum allowed batch size
# ids_batches = list(split_into_batches(df_id["_id"].tolist(), batch_size))
# title_embeddings_batches = list(split_into_batches(title_embeddings, batch_size))
# abstract_embeddings_batches = list(split_into_batches(abstract_embeddings, batch_size))
# title_metadatas_batches = list(split_into_batches(df_id[["title"]].to_dict(orient="records"), batch_size))
# abstract_metadatas_batches = list(split_into_batches(df_id[["abstract_str"]].to_dict(orient="records"), batch_size))

# for ids_batch, title_embeddings_batch, title_metadatas_batch in zip(ids_batches, title_embeddings_batches, title_metadatas_batches):
#     title_collection.add(ids=ids_batch, embeddings=title_embeddings_batch, metadatas=title_metadatas_batch)

# for ids_batch, abstract_embeddings_batch, abstract_metadatas_batch in zip(ids_batches, abstract_embeddings_batches, abstract_metadatas_batches):
#     abstract_collection.add(ids=ids_batch, embeddings=abstract_embeddings_batch, metadatas=abstract_metadatas_batch)


# 3. Similarity Computation and Re-ranking

In [57]:
# Function to compute similarity
def compute_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)

# Function to find similar texts
def find_similar_texts(user_input, embedding_column, top_n=30):
    user_embedding = vectorizer.transform([user_input]).toarray().squeeze()
    
    similarities = []
    for idx, row in df_id.iterrows():
        sim = compute_similarity(user_embedding, row[embedding_column])
        similarities.append((row['id'], sim))
    
    # Sort by similarity and get top N results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Function for re-ranking the results
def re_rank_results(similar_texts):
    # Combine similarity score with citation count for re-ranking
    re_ranked = []
    for id, similarity in similar_texts:
        citation_count = df_id[df_id["id"] == id]["citation_count"].values[0]
        # Example: Weighted combination of similarity and citation count
        re_ranked_score = 0.8 * similarity + 0.2 * (citation_count / 100)  # Normalize citation_count to 0-1 range
        re_ranked.append((id, re_ranked_score))
    
    # Sort by re-ranked score
    re_ranked.sort(key=lambda x: x[1], reverse=True)
    return re_ranked

# Updated function to find similar texts and re-rank them
def find_and_rerank_similar_texts(user_input, embedding_column, top_n=30):
    similar_texts = find_similar_texts(user_input, embedding_column, top_n)
    re_ranked_texts = re_rank_results(similar_texts)
    return re_ranked_texts


# . Recommendation Function

In [58]:
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Function to compute similarity
# def compute_similarity(embedding1, embedding2):
#     return cosine_similarity([embedding1], [embedding2])[0][0]

# # Function to find and rerank similar texts
# def find_and_rerank_similar_texts(user_query, top_n=5):
#     user_title_embedding = vectorizer.transform([user_query]).toarray().squeeze()
#     user_abstract_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    
#     similarities = []
#     for idx, row in df_id.iterrows():
#         title_sim = compute_similarity(user_title_embedding, row['title_embeddings'])
#         abstract_sim = compute_similarity(user_abstract_embedding, row['abstract_embeddings'])
#         combined_sim = 0.5 * title_sim + 0.5 * abstract_sim  # Equal weighting of title and abstract similarity
#         similarities.append((row['_id'], row['title'], combined_sim))
    
#     # Sort by combined similarity and get top N results
#     similarities.sort(key=lambda x: x[2], reverse=True)
#     return similarities[:top_n]

# # Function to recommend papers based on user input
# def recommend_articles(user_query, top_n=5):
#     # Find and rerank similar texts
#     similar_texts = find_and_rerank_similar_texts(user_query, top_n)
    
#     recommended_articles = []
#     for _id, title, score in similar_texts:
#         recommended_articles.append((_id, title, score))
    
#     return recommended_articles

# # Example usage
# user_query = "NLP for text analysis"
# recommended_articles = recommend_articles(user_query, top_n=5)

# # Display the recommended articles
# print("Recommended Articles:")
# for _id, title, score in recommended_articles:
#     print(f"ID: {_id}, Title: {title}, Similarity Score: {score}")


# New Codee

In [59]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np


# # Function to compute similarity
# def compute_similarity(embedding1, embedding2):
#     return cosine_similarity([embedding1], [embedding2])[0][0]

# # Function to compute keyword match score
# def compute_keyword_match_score(user_keywords, title):
#     title_words = set(title.lower().split())
#     matched_keywords = user_keywords.intersection(title_words)
#     return len(matched_keywords) / len(user_keywords) if user_keywords else 0

# # Function to find and rerank similar texts
# def find_and_rerank_similar_texts(user_query, top_n=5):
#     user_query_embedding = vectorizer.transform([user_query]).toarray().squeeze()
#     user_keywords = set(user_query.lower().split())
    
#     similarities = []
#     for idx, row in df_id.iterrows():
#         title_embedding = row['title_embeddings']
#         abstract_embedding = row['abstract_embeddings']
        
#         title_sim = compute_similarity(user_query_embedding, title_embedding)
#         abstract_sim = compute_similarity(user_query_embedding, abstract_embedding)
#         keyword_match_score = compute_keyword_match_score(user_keywords, row['title'])
        
#         # Combined similarity with weights (adjust weights as needed)
#         combined_sim = 0.4 * title_sim + 0.4 * abstract_sim + 0.2 * keyword_match_score
        
#         similarities.append((row['_id'], row['title'], combined_sim))
    
#     # Sort by combined similarity in descending order
#     similarities.sort(key=lambda x: x[2], reverse=True)
#     return similarities[:top_n]

# # Function to recommend papers based on user input
# def recommend_articles(user_query, top_n=5):
#     # Find and rerank similar texts
#     similar_texts = find_and_rerank_similar_texts(user_query, top_n)
    
#     recommended_articles = []
#     for _id, title, score in similar_texts:
#         recommended_articles.append((_id, title, score))
    
#     return recommended_articles

# # Example usage
# user_query = "NLP for text analysis"
# recommended_articles = recommend_articles(user_query, top_n=5)

# print(f"Recommended Articles for '{user_query}':")
# for rank, (_id, title, score) in enumerate(recommended_articles, 1):
#     print(f"Rank {rank}: ID: {_id}, Title: {title}, Similarity Score: {score:.4f}")


In [60]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to find and rerank similar texts using combined embeddings
def find_and_rerank_similar_texts(user_query, top_n=5):
    user_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    
    similarities = []
    for idx, row in df_id.iterrows():
        combined_embedding = row['combined_embeddings']
        
        # Compute cosine similarity between user embedding and combined embedding
        combined_sim = cosine_similarity([user_embedding], [combined_embedding])[0][0]
        
        similarities.append((row['id'], row['title'], combined_sim))
    
    # Sort by combined similarity in descending order
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]

# Function to recommend papers based on user input using combined embeddings
def recommend_articles(user_query, top_n=5):
    # Find and rerank similar texts using combined embeddings
    similar_texts = find_and_rerank_similar_texts(user_query, top_n)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
        
    
    return recommended_articles

# Example usage
user_queries = [
    " A new approach of 3D watermarking based on image segmentation"
]

for user_query in user_queries:
    print(f"Recommended Articles for '{user_query}':")
    recommended_articles = recommend_articles(user_query, top_n=5)

    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Similarity Score: {score:.4f}")
    print()  # Print empty line for separation


Recommended Articles for ' A new approach of 3D watermarking based on image segmentation':
Rank 1: ID: 67a59884-926e-4d8f-b638-c5140b91dcf2, Title: ['paraphras', 'identif', 'lexicosyntact', 'graph', 'subsumpt'], Similarity Score: 0.5292
Rank 2: ID: 505ea144-8353-456e-8adf-f0818f3bef0c, Title: ['teach', 'objectsfirst', 'introductori', 'comput', 'scienc'], Similarity Score: 0.4904
Rank 3: ID: 536eddb5-b1ac-45dc-9dc7-2734970c84d2, Title: ['differ', 'quantis', 'nois', 'shape', 'method', 'predict', 'audio', 'code'], Similarity Score: 0.4603
Rank 4: ID: 5ceab604-7649-4484-ac79-05bff7111aaf, Title: ['componentbas', 'approach', 'improv', 'modular', 'oscar'], Similarity Score: 0.4534
Rank 5: ID: 6926c0c1-ef07-40ab-9992-ca2346b890c0, Title: ['branchandcut', 'approach', 'gener', 'multipleproduct', 'assemblysystem', 'design', 'problem'], Similarity Score: 0.4507



In [61]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to find and rerank similar texts using combined embeddings
def find_and_rerank_similar_texts(user_query, top_n=5):
    user_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    
    similarities = []
    for idx, row in df_id.iterrows():
        combined_embedding = row['combined_embeddings']
        
        # Compute cosine similarity between user embedding and combined embedding
        combined_sim = cosine_similarity([user_embedding], [combined_embedding])[0][0]
        
        similarities.append((row['id'], row['title'], combined_sim))
    
    # Sort by combined similarity in descending order
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]

# Function to recommend papers based on user input using combined embeddings
def recommend_articles(user_query, top_n=5):
    # Find and rerank similar texts using combined embeddings
    similar_texts = find_and_rerank_similar_texts(user_query, top_n)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles


# Function to evaluate recommendations


# Example usage
user_queries = [
    "Bibliography on cyclostationarity"
]

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
        # stored_variable = (exact_match_id, exact_match_title)  # Store in variable
        # print(f"Stored Variable: {stored_variable}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, top_n=5)
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Similarity Score: {score:.4f}")

    # if exact_match_id is not None:
    #     accuracy = evaluate_accuracy(exact_match_id, recommended_articles)
    #     print(f"Accuracy: {accuracy:.4f}\n")  # Print empty line for separation
    # else:
    #     print(f"No exact title match found for '{user_query}' in df_id\n")
  


User Input Exact Title: 'Bibliography on cyclostationarity':
ID: 6d4c5b32-8e13-4022-b67f-1ace7ffc91d0, Title: Bibliography on cyclostationarity
Rank 1: ID: 4ab3735c-80f1-472d-b953-fa0557fed28b, Title: ['new', 'approach', 'watermark', 'base', 'imag', 'segment'], Similarity Score: 0.0000
Rank 2: ID: 4ab39729-af77-46f7-a662-16984fb9c1db, Title: ['attractor', 'neural', 'network', 'activitydepend', 'synaps', 'role', 'synapt', 'facilit'], Similarity Score: 0.0000
Rank 3: ID: 4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de, Title: ['character', 'balanc', 'episturmian', 'sequenc'], Similarity Score: 0.0000
Rank 4: ID: 4ab3a98c-3620-47ec-b578-884ecf4a6206, Title: ['explor', 'space', 'human', 'action'], Similarity Score: 0.0000
Rank 5: ID: 4ab3b585-82b4-4207-91dd-b6bce7e27c4e, Title: ['gener', 'upper', 'bound', 'minimum', 'distanc', 'psk', 'block', 'code'], Similarity Score: 0.0000


## This Part can be used for Jaccard_Score Pipline 2

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
import numpy as np

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to compute Jaccard similarity
def jaccard_similarity(query, document):
    query_set = set(query.split())
    document_set = set(document.split())
    intersection = query_set.intersection(document_set)
    union = query_set.union(document_set)
    if not union:  # Avoid division by zero
        return 0.0
    return len(intersection) / len(union)

# Function to find and rerank similar texts using Jaccard similarity
def find_and_rerank_similar_texts(user_query, df_id, top_n=5):
    similarities = []
    for idx, row in df_id.iterrows():
        document = row['title'] + " " + row['abstract']  # Combine title and abstract
        sim_score = jaccard_similarity(user_query, document)
        similarities.append((row['id'], row['title'], sim_score))
    
    # Sort by Jaccard similarity in descending order
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]

# Function to recommend papers based on user input using Jaccard similarity
def recommend_articles(user_query, df_id, top_n=5):
    # Find and rerank similar texts using Jaccard similarity
    similar_texts = find_and_rerank_similar_texts(user_query, df_id, top_n)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles

# Example usage
user_queries = [
    "Bibliography on cyclostationarity"
]

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
        # stored_variable = (exact_match_id, exact_match_title)  # Store in variable
        # print(f"Stored Variable: {stored_variable}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, df_id, top_n=5)
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Jaccard Similarity Score: {score:.4f}")


User Input Exact Title: 'Bibliography on cyclostationarity':
ID: 6d4c5b32-8e13-4022-b67f-1ace7ffc91d0, Title: Bibliography on cyclostationarity
Rank 1: ID: 5abb4a53-a24d-48d4-84ed-8f90633a30ba, Title: ['enterpriseori', 'iot', 'name', 'servic', 'agricultur', 'product', 'suppli', 'chain', 'manag'], Jaccard Similarity Score: 0.0222
Rank 2: ID: 69bcdef0-f057-43d4-8808-15e4e94f5038, Title: ['rainbow', 'turán', 'problem'], Jaccard Similarity Score: 0.0204
Rank 3: ID: 5a49ccd4-417f-4585-9d69-7bceb3e13f17, Title: ['predict', 'genet', 'drift', 'game'], Jaccard Similarity Score: 0.0182
Rank 4: ID: 6b7e733d-d80a-43c3-a722-caeee33c0374, Title: ['prototyp', 'multiroot', 'on'], Jaccard Similarity Score: 0.0169
Rank 5: ID: 5d17295b-c527-47ef-b8aa-b481ab1c946d, Title: ['implement', 'secur', 'ppon'], Jaccard Similarity Score: 0.0125


# Pipeline 3: WordNet using the Cosine Smilirty

In [71]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Shahzad
[nltk_data]     Ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Shahzad
[nltk_data]     Ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Shahzad
[nltk_data]     Ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Shahzad
[nltk_data]     Ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity

# Assuming nltk data is already downloaded in your environment

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to compute WordNet similarity between two words
def wordnet_similarity(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    if not synsets1 or not synsets2:  # If either word has no synsets
        return 0
    max_sim = max((syn1.path_similarity(syn2) or 0) for syn1 in synsets1 for syn2 in synsets2)
    return max_sim

# Function to compute similarity between two texts using WordNet
def text_similarity(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    if not words1 or not words2:  # If either text has no valid words
        return 0
    total_sim = 0
    for word1 in words1:
        word_sim = max(wordnet_similarity(word1, word2) for word2 in words2)
        total_sim += word_sim
    return total_sim / max(len(words1), len(words2))  # Normalize by the number of words

# Function to compute cosine similarity
def compute_cosine_similarity(user_query, df_id, vectorizer):
    user_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    cosine_similarities = []
    for idx, row in df_id.iterrows():
        combined_embedding = row['combined_embeddings']  # Assuming combined_embeddings is a precomputed field
        cosine_sim = cosine_similarity([user_embedding], [combined_embedding])[0][0]
        cosine_similarities.append((row['id'], row['title'], cosine_sim))
    return cosine_similarities

# Function to find and rerank similar texts using combined WordNet and cosine similarity
def find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    wordnet_similarities = []
    for idx, row in df_id.iterrows():
        document = row['title'] + " " + row['abstract']  # Combine title and abstract
        sim_score = text_similarity(user_query, document)
        wordnet_similarities.append((row['id'], row['title'], sim_score))
    
    cosine_similarities = compute_cosine_similarity(user_query, df_id, vectorizer)
    
    combined_similarities = []
    for (id1, title1, wordnet_sim), (id2, title2, cosine_sim) in zip(wordnet_similarities, cosine_similarities):
        assert id1 == id2  # Ensure that the IDs match
        combined_sim = alpha * wordnet_sim + (1 - alpha) * cosine_sim
        combined_similarities.append((id1, title1, combined_sim))
    
    # Sort by combined similarity in descending order
    combined_similarities.sort(key=lambda x: x[2], reverse=True)
    return combined_similarities[:top_n]

# Function to recommend papers based on user input using combined similarities
def recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    # Find and rerank similar texts using combined similarities
    similar_texts = find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n, alpha)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles

# Example usage
user_queries = [
    "Bibliography on cyclostationarity"
]

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5)
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Combined Similarity Score: {score:.4f}")


# Pipline 4: Jacord smilirty using the wordNet

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import jaccard_score
import numpy as np
import pandas as pd

# Assuming nltk data is already downloaded in your environment

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to compute WordNet similarity between two words
def wordnet_similarity(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    if not synsets1 or not synsets2:  # If either word has no synsets
        return 0
    max_sim = max((syn1.path_similarity(syn2) or 0) for syn1 in synsets1 for syn2 in synsets2)
    return max_sim

# Function to compute similarity between two texts using WordNet
def text_similarity(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    if not words1 or not words2:  # If either text has no valid words
        return 0
    total_sim = 0
    for word1 in words1:
        word_sim = max(wordnet_similarity(word1, word2) for word2 in words2)
        total_sim += word_sim
    return total_sim / max(len(words1), len(words2))  # Normalize by the number of words

# Function to compute Jaccard similarity
def compute_jaccard_similarity(user_query, df_id, vectorizer):
    user_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    jaccard_similarities = []
    for idx, row in df_id.iterrows():
        document_embedding = row['combined_embeddings']  # Assuming combined_embeddings is a precomputed field
        jaccard_sim = jaccard_score(user_embedding, document_embedding, average='binary')
        jaccard_similarities.append((row['id'], row['title'], jaccard_sim))
    return jaccard_similarities

# Function to find and rerank similar texts using combined WordNet and Jaccard similarity
def find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    wordnet_similarities = []
    for idx, row in df_id.iterrows():
        document = row['title'] + " " + row['abstract']  # Combine title and abstract
        sim_score = text_similarity(user_query, document)
        wordnet_similarities.append((row['id'], row['title'], sim_score))
    
    jaccard_similarities = compute_jaccard_similarity(user_query, df_id, vectorizer)
    
    combined_similarities = []
    for (id1, title1, wordnet_sim), (id2, title2, jaccard_sim) in zip(wordnet_similarities, jaccard_similarities):
        assert id1 == id2  # Ensure that the IDs match
        combined_sim = alpha * wordnet_sim + (1 - alpha) * jaccard_sim
        combined_similarities.append((id1, title1, combined_sim))
    
    # Sort by combined similarity in descending order
    combined_similarities.sort(key=lambda x: x[2], reverse=True)
    return combined_similarities[:top_n]

# Function to recommend papers based on user input using combined similarities
def recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    # Find and rerank similar texts using combined similarities
    similar_texts = find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n, alpha)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles

# Example usage
user_queries = [
    "Bibliography on cyclostationarity"
]

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5)
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Combined Similarity Score: {score:.4f}")


# Pipline 5: SentiWordNet using the Cosine Smilirty

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to compute SentiWordNet sentiment score for a word
def sentiwordnet_score(word):
    synsets = wn.synsets(word)
    if not synsets:
        return 0, 0, 0  # No sentiment score if no synsets are found

    pos_score, neg_score, obj_score = 0, 0, 0
    for synset in synsets:
        senti_synset = swn.senti_synset(synset.name())
        pos_score += senti_synset.pos_score()
        neg_score += senti_synset.neg_score()
        obj_score += senti_synset.obj_score()
    return pos_score, neg_score, obj_score

# Function to compute sentiment similarity between two words
def sentiment_similarity(word1, word2):
    pos1, neg1, obj1 = sentiwordnet_score(word1)
    pos2, neg2, obj2 = sentiwordnet_score(word2)
    # Similarity based on positive and negative sentiment scores
    return (pos1 * pos2 + neg1 * neg2) / (np.sqrt((pos1 ** 2 + neg1 ** 2) * (pos2 ** 2 + neg2 ** 2)) + 1e-10)

# Function to compute similarity between two texts using SentiWordNet
def text_similarity(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    if not words1 or not words2:  # If either text has no valid words
        return 0
    total_sim = 0
    for word1 in words1:
        word_sim = max(sentiment_similarity(word1, word2) for word2 in words2)
        total_sim += word_sim
    return total_sim / max(len(words1), len(words2))  # Normalize by the number of words

# Function to compute Cosine similarity
def compute_cosine_similarity(user_query, df_id, vectorizer):
    user_embedding = vectorizer.transform([user_query]).toarray()
    cosine_similarities = []
    for idx, row in df_id.iterrows():
        document_embedding = np.array(row['combined_embeddings']).reshape(1, -1)  # Assuming combined_embeddings is a precomputed field
        cosine_sim = cosine_similarity(user_embedding, document_embedding)[0][0]
        cosine_similarities.append((row['id'], row['title'], cosine_sim))
    return cosine_similarities

# Function to find and rerank similar texts using combined SentiWordNet and Cosine similarity
def find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    sentiwordnet_similarities = []
    for idx, row in df_id.iterrows():
        document = row['title'] + " " + row['abstract']  # Combine title and abstract
        sim_score = text_similarity(user_query, document)
        sentiwordnet_similarities.append((row['id'], row['title'], sim_score))
    
    cosine_similarities = compute_cosine_similarity(user_query, df_id, vectorizer)
    
    combined_similarities = []
    for (id1, title1, sentiwordnet_sim), (id2, title2, cosine_sim) in zip(sentiwordnet_similarities, cosine_similarities):
        assert id1 == id2  # Ensure that the IDs match
        combined_sim = alpha * sentiwordnet_sim + (1 - alpha) * cosine_sim
        combined_similarities.append((id1, title1, combined_sim))
    
    # Sort by combined similarity in descending order
    combined_similarities.sort(key=lambda x: x[2], reverse=True)
    return combined_similarities[:top_n]

# Function to recommend papers based on user input using combined similarities
def recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    # Find and rerank similar texts using combined similarities
    similar_texts = find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n, alpha)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles

# Example usage
user_queries = [
    "Bibliography on cyclostationarity"
]

# Assuming 'vectorizer' is already defined and fitted
vectorizer = TfidfVectorizer()  # Replace with your actual vectorizer

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5)
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Combined Similarity Score: {score:.4f}")


## Pipline 6: SentiwordNet using the Jaccord_score

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from sklearn.metrics import jaccard_score
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to compute SentiWordNet sentiment score for a word
def sentiwordnet_score(word):
    synsets = wn.synsets(word)
    if not synsets:
        return 0, 0, 0  # No sentiment score if no synsets are found

    pos_score, neg_score, obj_score = 0, 0, 0
    for synset in synsets:
        senti_synset = swn.senti_synset(synset.name())
        pos_score += senti_synset.pos_score()
        neg_score += senti_synset.neg_score()
        obj_score += senti_synset.obj_score()
    return pos_score, neg_score, obj_score

# Function to compute sentiment similarity between two words
def sentiment_similarity(word1, word2):
    pos1, neg1, obj1 = sentiwordnet_score(word1)
    pos2, neg2, obj2 = sentiwordnet_score(word2)
    # Similarity based on positive and negative sentiment scores
    return (pos1 * pos2 + neg1 * neg2) / (np.sqrt((pos1 ** 2 + neg1 ** 2) * (pos2 ** 2 + neg2 ** 2)) + 1e-10)

# Function to compute similarity between two texts using SentiWordNet
def text_similarity(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    if not words1 or not words2:  # If either text has no valid words
        return 0
    total_sim = 0
    for word1 in words1:
        word_sim = max(sentiment_similarity(word1, word2) for word2 in words2)
        total_sim += word_sim
    return total_sim / max(len(words1), len(words2))  # Normalize by the number of words

# Function to compute Jaccard similarity
def compute_jaccard_similarity(user_query, df_id, vectorizer):
    user_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    jaccard_similarities = []
    for idx, row in df_id.iterrows():
        document_embedding = row['combined_embeddings']  # Assuming combined_embeddings is a precomputed field
        jaccard_sim = jaccard_score(user_embedding, document_embedding, average='binary')
        jaccard_similarities.append((row['id'], row['title'], jaccard_sim))
    return jaccard_similarities

# Function to find and rerank similar texts using combined SentiWordNet and Jaccard similarity
def find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    sentiwordnet_similarities = []
    for idx, row in df_id.iterrows():
        document = row['title'] + " " + row['abstract']  # Combine title and abstract
        sim_score = text_similarity(user_query, document)
        sentiwordnet_similarities.append((row['id'], row['title'], sim_score))
    
    jaccard_similarities = compute_jaccard_similarity(user_query, df_id, vectorizer)
    
    combined_similarities = []
    for (id1, title1, sentiwordnet_sim), (id2, title2, jaccard_sim) in zip(sentiwordnet_similarities, jaccard_similarities):
        assert id1 == id2  # Ensure that the IDs match
        combined_sim = alpha * sentiwordnet_sim + (1 - alpha) * jaccard_sim
        combined_similarities.append((id1, title1, combined_sim))
    
    # Sort by combined similarity in descending order
    combined_similarities.sort(key=lambda x: x[2], reverse=True)
    return combined_similarities[:top_n]

# Function to recommend papers based on user input using combined similarities
def recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5):
    # Find and rerank similar texts using combined similarities
    similar_texts = find_and_rerank_similar_texts(user_query, df_id, vectorizer, top_n, alpha)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles

# Example usage
user_queries = [
    "Bibliography on cyclostationarity"
]

# Assuming 'vectorizer' is already defined and fitted
vectorizer = TfidfVectorizer()  # Replace with your actual vectorizer

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, df_id, vectorizer, top_n=5, alpha=0.5)
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Combined Similarity Score: {score:.4f}")


## Pipline 7: LLM using the Cosine Smilirty

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import chromadb
from chromadb.config import Settings
import random

# Load the SciBERT model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

# Assuming df_id1 is your DataFrame with 'title' and 'abstract' columns

# Combine title and abstract
df_id1['combined_text'] = df_id1['title'] + ' ' + df_id1['abstract']

# Get embeddings for combined text
df_id1['combined_embeddings'] = df_id1['combined_text'].apply(lambda x: get_embeddings(x).squeeze().numpy())

# Convert numpy arrays to lists
df_id1['combined_embeddings'] = df_id1['combined_embeddings'].apply(lambda x: x.tolist())

# Convert combined text to string for metadata
df_id1['combined_str'] = df_id1['combined_text'].apply(lambda x: ' '.join(x.split()))

# Connect to Chroma and create a collection
client = chromadb.Client(Settings())
combined_collection = client.create_collection("combined_embeddings")

# Add embeddings to the collection
ids = df_id1["id"].tolist()
combined_embeddings = df_id1["combined_embeddings"].tolist()
combined_metadatas = df_id1[["combined_str"]].to_dict(orient="records")

combined_collection.add(ids=ids, embeddings=combined_embeddings, metadatas=combined_metadatas)

# Function to compute similarity
def compute_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)

# Function to find similar texts
def find_similar_texts(user_input, top_n=5):
    user_embedding = get_embeddings(user_input).squeeze().numpy().tolist()
    
    similarities = []
    for idx, row in df_id1.iterrows():
        sim = compute_similarity(user_embedding, row['combined_embeddings'])
        similarities.append((row['id'], sim))
    
    # Sort by similarity and get top N results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Function for re-ranking the results
def re_rank_results(similar_texts):
    # Your re-ranking logic here (if any)
    return similar_texts  # For simplicity, returning the same list

# Updated function to find similar texts and re-rank them
def find_and_rerank_similar_texts(user_input, top_n=5):
    similar_texts = find_similar_texts(user_input, top_n)
    re_ranked_texts = re_rank_results(similar_texts)
    return re_ranked_texts

# Function to recommend papers (example random recommendation)
def recommend_papers(user_input, num_recommendations=5):
    recommended_papers = random.sample(df_id1["id"].tolist(), num_recommendations)
    return recommended_papers

# Function to calculate BERTScore for recommended papers
def calculate_bertscore_for_recommendations(user_input, recommended_papers):
    candidates = [user_input] * len(recommended_papers)
    references = [df_id1[df_id1["id"] == paper_id]["combined_str"].values[0] for paper_id in recommended_papers]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1_score": F1.mean().item()
    }

# Function to find exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Example usage
user_input = "recommendation system based on semantic analysis"
similar_texts = find_and_rerank_similar_texts(user_input)
recommended_papers = recommend_papers(user_input)

# Display the results for combined text
print("Similar Combined Texts:")
for id, similarity in similar_texts:
    combined_text = df_id1[df_id1["id"] == id]["combined_text"].values[0]
    print(f"Combined Text: {combined_text}, Similarity: {similarity}")

# Display recommended papers
print("Recommended Papers:")
for paper_id in recommended_papers:
    combined_text = df_id1[df_id1["id"] == paper_id]["combined_text"].values[0]
    print(f"Paper ID: {paper_id}, Combined Text: {combined_text}")

# Calculate BERTScore for recommended papers
bert_metrics_recommended_papers = calculate_bertscore_for_recommendations(user_input, recommended_papers)

# Display the BERTScore for recommended papers
print("BERTScore Metrics for Recommended Papers:")
print(f"Precision: {bert_metrics_recommended_papers['precision']:.2f}")
print(f"Recall: {bert_metrics_recommended_papers['recall']:.2f}")
print(f"F1 Score: {bert_metrics_recommended_papers['f1_score']:.2f}")

# Example usage of find_exact_title_match function
title_query = "Example Title 1"
matched_id, matched_title = find_exact_title_match(title_query, df_id1)
print(f"Exact match - ID: {matched_id}, Title: {matched_title}")


In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import chromadb
from chromadb.config import Settings

# Load the SciBERT model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

# Assuming df is your DataFrame with 'title' and 'abstract' columns
df = pd.DataFrame({
    'title': ["Example Title 1", "Example Title 2"],
    'abstract': ["This is an example abstract 1.", "This is an example abstract 2."],
    '_id': [1, 2]
})

# Get embeddings for titles and abstracts
df_id1['title_embeddings'] = df_id1['title'].apply(lambda x: get_embeddings(x).squeeze().numpy())
df_id1['abstract_embeddings'] = df_id1['abstract'].apply(lambda x: get_embeddings(x).squeeze().numpy())

# Convert numpy arrays to lists
df_id1['title_embeddings'] = df_id1['title_embeddings'].apply(lambda x: x.tolist())
df_id1['abstract_embeddings'] = df_id1['abstract_embeddings'].apply(lambda x: x.tolist())

# Convert lists in 'abstract' to strings for metadata
df['abstract_str'] = df['abstract'].apply(lambda x: ' '.join(x.split()))

# Connect to Chroma and create collections
client = chromadb.Client(Settings())
title_collection = client.create_collection("title_embeddings")
abstract_collection = client.create_collection("abstract_embeddings")

# Add embeddings to collections
ids = df["_id"].tolist()
title_embeddings = df["title_embeddings"].tolist()
abstract_embeddings = df["abstract_embeddings"].tolist()
title_metadatas = df[["title"]].to_dict(orient="records")
abstract_metadatas = df[["abstract_str"]].to_dict(orient="records")

title_collection.add(ids=ids, embeddings=title_embeddings, metadatas=title_metadatas)
abstract_collection.add(ids=ids, embeddings=abstract_embeddings, metadatas=abstract_metadatas)

# Function to compute similarity
def compute_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)

# Function to find similar texts
def find_similar_texts(user_input, embedding_column, top_n=5):
    user_embedding = get_embeddings(user_input).squeeze().numpy().tolist()
    
    similarities = []
    for idx, row in df.iterrows():
        sim = compute_similarity(user_embedding, row[embedding_column])
        similarities.append((row['_id'], sim))
    
    # Sort by similarity and get top N results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Function for re-ranking the results
def re_rank_results(similar_texts):
    # Your re-ranking logic here (if any)
    return similar_texts  # For simplicity, returning the same list

# Updated function to find similar texts and re-rank them
def find_and_rerank_similar_texts(user_input, embedding_column, top_n=5):
    similar_texts = find_similar_texts(user_input, embedding_column, top_n)
    re_ranked_texts = re_rank_results(similar_texts)
    return re_ranked_texts

# Function to recommend papers (example random recommendation)
def recommend_papers(user_input, num_recommendations=5):
    recommended_papers = random.sample(df["_id"].tolist(), num_recommendations)
    return recommended_papers

# Function to calculate BERTScore for recommended papers
def calculate_bertscore_for_recommendations(user_input, recommended_papers):
    candidates = [user_input] * len(recommended_papers)
    references = [df[df["_id"] == paper_id]["abstract_str"].values[0] for paper_id in recommended_papers]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1_score": F1.mean().item()
    }

# Example usage
user_input = "recommendation system based on semantic analysis"
similar_titles = find_and_rerank_similar_texts(user_input, 'title_embeddings')
similar_abstracts = find_and_rerank_similar_texts(user_input, 'abstract_embeddings')
recommended_papers = recommend_papers(user_input)

# Display the results for titles
print("Similar Titles:")
for _id, similarity in similar_titles:
    title = df[df["_id"] == _id]["title"].values[0]
    print(f"Title: {title}, Similarity: {similarity}")

# Display the results for abstracts
print("Similar Abstracts:")
for _id, similarity in similar_abstracts:
    title = df[df["_id"] == _id]["title"].values[0]
    print(f"Title: {title}, Similarity: {similarity}")

# Display recommended papers
print("Recommended Papers:")
for paper_id in recommended_papers:
    title = df[df["_id"] == paper_id]["title"].values[0]
    print(f"Paper ID: {paper_id}, Title: {title}")

# Calculate BERTScore for recommended papers
bert_metrics_recommended_papers = calculate_bertscore_for_recommendations(user_input, recommended_papers)

# Display the BERTScore for recommended papers
print("BERTScore Metrics for Recommended Papers:")
print(f"Precision: {bert_metrics_recommended_papers['precision']:.2f}")
print(f"Recall: {bert_metrics_recommended_papers['recall']:.2f}")
print(f"F1 Score: {bert_metrics_recommended_papers['f1_score']:.2f}")


In [65]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Function to find the ID and title based on an exact title match
def find_exact_title_match(title_query, df_id1):
    for idx, row in df_id1.iterrows():
        if row['title'].strip().lower() == title_query.strip().lower():
            return row['id'], row['title']
    return None, None  # Return None if no exact match is found

# Function to find and rerank similar texts using combined embeddings
def find_and_rerank_similar_texts(user_query, top_n=5):
    user_embedding = vectorizer.transform([user_query]).toarray().squeeze()
    
    similarities = []
    for idx, row in df_id.iterrows():
        combined_embedding = row['combined_embeddings']
        
        # Compute cosine similarity between user embedding and combined embedding
        combined_sim = cosine_similarity([user_embedding], [combined_embedding])[0][0]
        
        similarities.append((row['id'], row['title'], combined_sim))
    
    # Sort by combined similarity in descending order
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]

# Function to recommend papers based on user input using combined embeddings
def recommend_articles(user_query, top_n=5):
    # Find and rerank similar texts using combined embeddings
    similar_texts = find_and_rerank_similar_texts(user_query, top_n)
    
    recommended_articles = []
    for id, title, score in similar_texts:
        recommended_articles.append((id, title, score))
    
    return recommended_articles

# Function to evaluate recommendations based on citation relationship
def evaluate_recommendations(user_id, recommendation_ids):
    user_references = df_referce[df_referce['id'] == user_id]['references'].values
    if len(user_references) > 0:
        user_references = user_references[0]
        if pd.isna(user_references):
            user_references = []
        elif isinstance(user_references, float):
            user_references = []
        else:
            user_references = eval(user_references) if isinstance(user_references, str) else user_references
    else:
        user_references = []

    evaluation_matrix = []
    for rec_id in recommendation_ids:
        rec_references = df_referce[df_referce['id'] == rec_id]['references'].values
        if len(rec_references) > 0:
            rec_references = rec_references[0]
            if pd.isna(rec_references):
                rec_references = []
            elif isinstance(rec_references, float):
                rec_references = []
            else:
                rec_references = eval(rec_references) if isinstance(rec_references, str) else rec_references
        else:
            rec_references = []

        in_user_references = rec_id in user_references
        user_id_in_rec_references = user_id in rec_references
        
        evaluation_matrix.append((rec_id, in_user_references, user_id_in_rec_references))
    
    # Create DataFrame for evaluation matrix
    df_evaluation = pd.DataFrame(evaluation_matrix, columns=['Recommended_ID', 'In_User_References', 'User_ID_in_Rec_References'])
    
    # Calculate metrics
    true_positives = sum(df_evaluation['In_User_References'] | df_evaluation['User_ID_in_Rec_References'])
    false_positives = len(df_evaluation) - true_positives
    total_relevant_items = len(set(user_references).union(set(recommendation_ids)))
    false_negatives = total_relevant_items - true_positives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

# Example usage
user_queries = [
    "A new approach of 3D watermarking based on image segmentation"
]

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
        # stored_variable = (exact_match_id, exact_match_title)  # Store in variable
        # print(f"Stored Variable: {stored_variable}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, top_n=5)
    recommendation_ids = [id for id, title, score in recommended_articles]
    
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Similarity Score: {score:.4f}")

    # Evaluate recommendations if there are recommended articles
    if recommendation_ids:
        precision, recall, f1_score = evaluate_recommendations(exact_match_id, recommendation_ids)
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
    else:
        print("No recommendations to evaluate.\n")


User Input Exact Title: 'A new approach of 3D watermarking based on image segmentation':
ID: 4ab3735c-80f1-472d-b953-fa0557fed28b, Title: A new approach of 3D watermarking based on image segmentation
Rank 1: ID: 67a59884-926e-4d8f-b638-c5140b91dcf2, Title: ['paraphras', 'identif', 'lexicosyntact', 'graph', 'subsumpt'], Similarity Score: 0.5292
Rank 2: ID: 505ea144-8353-456e-8adf-f0818f3bef0c, Title: ['teach', 'objectsfirst', 'introductori', 'comput', 'scienc'], Similarity Score: 0.4904
Rank 3: ID: 536eddb5-b1ac-45dc-9dc7-2734970c84d2, Title: ['differ', 'quantis', 'nois', 'shape', 'method', 'predict', 'audio', 'code'], Similarity Score: 0.4603
Rank 4: ID: 5ceab604-7649-4484-ac79-05bff7111aaf, Title: ['componentbas', 'approach', 'improv', 'modular', 'oscar'], Similarity Score: 0.4534
Rank 5: ID: 6926c0c1-ef07-40ab-9992-ca2346b890c0, Title: ['branchandcut', 'approach', 'gener', 'multipleproduct', 'assemblysystem', 'design', 'problem'], Similarity Score: 0.4507


SyntaxError: '[' was never closed (<string>, line 1)

In [ ]:
import pandas as pd
import re

# Function to clean and format references
def format_references(references):
    """
    Clean and format references data.
    - Remove extra brackets at the start and end
    - Strip extra whitespace
    - Split by commas
    """
    if isinstance(references, str):
        # Remove extra brackets and whitespace
        cleaned_references = re.sub(r'^\[|\]$', '', references).strip()
        # Split by commas and remove any extra whitespace
        reference_list = [ref.strip() for ref in cleaned_references.split(',') if ref.strip()]
        return reference_list
    elif isinstance(references, list):
        # If references are already a list, ensure no unwanted brackets or whitespace
        return [ref.strip() for ref in references if ref.strip()]
    return []

# Function to evaluate recommendations based on citation relationship
def evaluate_recommendations(user_id, recommendation_ids):
    user_references = df_referce[df_referce['id'] == user_id]['references'].values
    if len(user_references) > 0:
        user_references = user_references[0]
        user_references = format_references(user_references)
    else:
        user_references = []

    evaluation_matrix = []
    for rec_id in recommendation_ids:
        rec_references = df_referce[df_referce['id'] == rec_id]['references'].values
        if len(rec_references) > 0:
            rec_references = rec_references[0]
            rec_references = format_references(rec_references)
        else:
            rec_references = []

        in_user_references = rec_id in user_references
        user_id_in_rec_references = user_id in rec_references
        
        evaluation_matrix.append((rec_id, in_user_references, user_id_in_rec_references))
    
    # Create DataFrame for evaluation matrix
    df_evaluation = pd.DataFrame(evaluation_matrix, columns=['Recommended_ID', 'In_User_References', 'User_ID_in_Rec_References'])
    
    # Calculate metrics
    true_positives = sum(df_evaluation['In_User_References'] | df_evaluation['User_ID_in_Rec_References'])
    false_positives = len(df_evaluation) - true_positives
    total_relevant_items = len(set(user_references).union(set(recommendation_ids)))
    false_negatives = total_relevant_items - true_positives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

# Example usage
user_queries = [
    "A new approach of 3D watermarking based on image segmentation"
]

for user_query in user_queries:
    print(f"User Input Exact Title: '{user_query}':")
    
    # Check for exact title match in df_id1
    exact_match_id, exact_match_title = find_exact_title_match(user_query, df_id1)
    
    if exact_match_id is not None:
        print(f"ID: {exact_match_id}, Title: {exact_match_title}")
        # stored_variable = (exact_match_id, exact_match_title)  # Store in variable
        # print(f"Stored Variable: {stored_variable}")
    else:
        print(f"No exact title match found in df_id1 for '{user_query}'")
    
    # Proceed with recommendation if no exact match found
    recommended_articles = recommend_articles(user_query, top_n=5)
    recommendation_ids = [id for id, title, score in recommended_articles]
    
    for rank, (id, title, score) in enumerate(recommended_articles, 1):
        print(f"Rank {rank}: ID: {id}, Title: {title}, Similarity Score: {score:.4f}")

    # Evaluate recommendations if there are recommended articles
    if recommendation_ids:
        precision, recall, f1_score = evaluate_recommendations(exact_match_id, recommendation_ids)
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
    else:
        print("No recommendations to evaluate.\n")


In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.spatial.distance import cosine
# import chromadb
# import numpy as np
# from chromadb.config import Settings

# # Load data

# # Ensure the 'title' and 'abstract' columns contain strings
# df_id['title'] = df_id['title'].astype(str)
# df_id['abstract'] = df_id['abstract'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

# # Example additional metadata for re-ranking
# df_id['citation_count'] = df_id['n_citation']  # Use actual citation counts

# # Create TF-IDF vectorizer
# vectorizer = TfidfVectorizer(max_features=768)  # Adjust the max_features based on your needs

# # Fit and transform the title and abstract columns
# df_id['title_embeddings'] = list(vectorizer.fit_transform(df_id['title']).toarray())
# df_id['abstract_embeddings'] = list(vectorizer.fit_transform(df_id['abstract']).toarray())

# # Convert lists in 'abstract' to strings for metadata
# df_id['abstract_str'] = df_id['abstract']

# # Ensure IDs are strings
# df_id['_id'] = df_id['_id'].astype(str)

# # Connect to Chroma and create collections
# client = chromadb.Client(Settings())
# title_collection = client.create_collection("title_embeddings")
# abstract_collection = client.create_collection("abstract_embeddings")

# # Add embeddings to collections
# ids = df_id["_id"].tolist()
# title_embeddings = df_id["title_embeddings"].tolist()
# abstract_embeddings = df_id["abstract_embeddings"].tolist()
# title_metadatas = df_id[["title"]].to_dict(orient="records")
# abstract_metadatas = df_id[["abstract_str"]].to_dict(orient="records")

# title_collection.add(ids=ids, embeddings=title_embeddings, metadatas=title_metadatas)
# abstract_collection.add(ids=ids, embeddings=abstract_embeddings, metadatas=abstract_metadatas)

# # Function to compute similarity
# def compute_similarity(embedding1, embedding2):
#     return 1 - cosine(embedding1, embedding2)

# # Function to find similar texts
# def find_similar_texts(user_input, embedding_column, top_n=5):
#     user_embedding = vectorizer.transform([user_input]).toarray().squeeze()
    
#     similarities = []
#     for idx, row in df_id.iterrows():
#         sim = compute_similarity(user_embedding, row[embedding_column])
#         similarities.append((row['_id'], sim))
    
#     # Sort by similarity and get top N results
#     similarities.sort(key=lambda x: x[1], reverse=True)
#     return similarities[:top_n]

# # Function for re-ranking the results
# def re_rank_results(similar_texts):
#     # Combine similarity score with citation count for re-ranking
#     re_ranked = []
#     for _id, similarity in similar_texts:
#         citation_count = df_id[df_id["_id"] == _id]["citation_count"].values[0]
#         # Example: Weighted combination of similarity and citation count
#         re_ranked_score = 0.8 * similarity + 0.2 * (citation_count / 100)  # Normalize citation_count to 0-1 range
#         re_ranked.append((_id, re_ranked_score))
    
#     # Sort by re-ranked score
#     re_ranked.sort(key=lambda x: x[1], reverse=True)
#     return re_ranked

# # Updated function to find similar texts and re-rank them
# def find_and_rerank_similar_texts(user_input, embedding_column, top_n=5):
#     similar_texts = find_similar_texts(user_input, embedding_column, top_n)
#     re_ranked_texts = re_rank_results(similar_texts)
#     return re_ranked_texts

# # Function to recommend papers (example random recommendation)
# def recommend_papers(user_input, num_recommendations=5):
#     recommended_papers = df_id["_id"].tolist()[:num_recommendations]  # Simplified for demonstration
#     return recommended_papers

# # Example usage
# user_input = "recommendation system based on semantic analysis"
# similar_titles = find_and_rerank_similar_texts(user_input, 'title_embeddings')
# similar_abstracts = find_and_rerank_similar_texts(user_input, 'abstract_embeddings')
# recommended_papers = recommend_papers(user_input)

# # Display the results for titles
# print("Similar Titles:")
# for _id, similarity in similar_titles:
#     title = df_id[df_id["_id"] == _id]["title"].values[0]
#     print(f"Title: {title}, Similarity: {similarity}")

# # Display the results for abstracts
# print("Similar Abstracts:")
# for _id, similarity in similar_abstracts:
#     title = df_id[df_id["_id"] == _id]["title"].values[0]
#     print(f"Title: {title}, Similarity: {similarity}")

# # Display recommended papers
# print("Recommended Papers:")
# for paper_id in recommended_papers:
#     title = df_id[df_id["_id"] == paper_id]["title"].values[0]
#     print(f"Paper ID: {paper_id}, Title: {title}")

# # Function to recommend articles based on user query
# def recommend_articles(user_query, top_n=5):
#     # Transform the user query into a TF-IDF embedding
#     query_embedding = vectorizer.transform([user_query]).toarray().squeeze()

#     # Retrieve similar documents from the collection
#     docs = title_collection.query(embeddings=[query_embedding], n_results=top_n)

#     recommended_articles = []
#     for doc in docs['documents']:
#         if doc is None or 'metadatas' not in doc or 'embeddings' not in doc:
#             continue  # Skip if document metadata or embeddings are missing

#         metadata = doc['metadatas'][0]
#         embedding = np.array(doc['embeddings'][0])

#         # Ensure embedding is a 1D array
#         if len(embedding.shape) == 1:
#             similarity_score = 1 - cosine(query_embedding, embedding)
#             recommended_articles.append((metadata['title'], similarity_score))

#     # Sort the recommended articles by similarity score
#     recommended_articles.sort(key=lambda x: x[1], reverse=True)
#     return recommended_articles[:top_n]

# # Example usage
# user_query = "Deep learning for natural language processing"
# recommended_articles = recommend_articles(user_query, top_n=5)

# # Display the recommended articles
# print("Recommended Articles:")
# for title, score in recommended_articles:
#     print(f"Title: {title}, Similarity Score: {score}")


In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from chromadb.config import Settings
# from chromadb import Client
# import chromadb


# #
# # Combine 'clean_title' and 'clean_abstract' for embedding
# df_id['combined_text'] = df_id['clean_title'].fillna('') + ' ' + df_id['clean_abstract'].fillna('')

# # Initialize the TF-IDF vectorizer
# vectorizer = TfidfVectorizer(max_features=5000)
# X = vectorizer.fit_transform(df_id['combined_text'])

# # Initialize the ChromaDB client
# client = chromadb.Client()

# # Create or connect to a Chroma collection
# collection_name = "research_papers2"
# if collection_name not in client.list_collections():
#     client.create_collection(name=collection_name)
# collection = client.get_collection(name=collection_name)

# # Store the embeddings in the Chroma vector database
# for idx, row in df_id.iterrows():
#         vector = X[idx].toarray().flatten().tolist()
#         doc_id = str(row['_id'])  # Ensure the ID is a string
#         title_str = ' '.join(row['title']) if isinstance(row['title'], list) else str(row['title'])
#         abstract_str = ' '.join(row['abstract']) if isinstance(row['abstract'], list) else str(row['abstract'])
#         collection.add(ids=[doc_id], embeddings=[vector], metadatas=[{
#         "title": title_str,
#         "abstract": abstract_str
#     }])

# print("Embeddings successfully stored in the Chroma vector database.")

In [ ]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from chromadb import Client
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming df_id, vectorizer, and collection are already defined from your previous code

# # Function to recommend top articles based on a user query
# def recommend_articles(user_query, top_n=5):
#     # Vectorize the user query using the existing vectorizer
#     user_query_vector = vectorizer.transform([user_query]).toarray().flatten()

#     # Initialize a list to store document embeddings and metadata
#     documents = []

#     # Iterate over documents in the collection
#     for idx in range(len(df_id)):
#         doc_id = df_id.iloc[idx]['_id']
#         doc = collection.get([str(doc_id)])
        
#         # Debug print to inspect document details
#         print(f"Processing document ID: {doc_id}")
#         print(doc)
        
#         if not doc['ids']:
#             continue  # Skip if document ID is not found

#         if not doc['metadatas'] or not doc['embeddings']:
#             continue  # Skip if metadata or embeddings are not found
        
#         metadata = doc['metadatas'][0]
#         embedding = doc['embeddings'][0]

#         # Convert embedding to numpy array if necessary
#         if isinstance(embedding, str):
#             embedding = np.fromstring(embedding.strip('[]'), dtype=float, sep=',')
        
#         embedding = np.array(embedding)

#         # Ensure embedding is a 1D array
#         if len(embedding.shape) == 1:
#             embedding = np.expand_dims(embedding, axis=0)
        
#         # Calculate cosine similarity between the user query vector and the stored vector
#         similarity = cosine_similarity([user_query_vector], embedding)[0][0]
#         documents.append((metadata, similarity))

#     # Rerank the results based on similarity
#     documents.sort(key=lambda x: x[1], reverse=True)

#     # Recommend the top N articles
#     top_articles = documents[:top_n]
#     for rank, (article, similarity) in enumerate(top_articles, 1):
#         print(f"Rank {rank}:")
#         print(f"Title: {article['title']}")
#         print(f"Abstract: {article['abstract']}")
#         print(f"Similarity: {similarity:.4f}")
#         print()

# # Example usage
# user_query = "Deep learning for natural language processing"
# recommend_articles(user_query, top_n=5)



In [ ]:
# from gensim.models import Word2Vec
# from nltk.tokenize import word_tokenize

# # Train Word2Vec model
# model = Word2Vec(df_id['lemmatized_title'], vector_size=100, window=5, min_count=1, workers=4)
# model1 = Word2Vec(df_id['lemmatized_abstract'], vector_size=100, window=5, min_count=1, workers=10)

# # Function to get the vector representation of a sentence
# def get_sentence_vector(tokens):
#     vector_sum = sum(model.wv[token] for token in tokens if token in model.wv)
#     return vector_sum / len(tokens)

#     vector_sum1 = sum(model1.wv[token] for token in tokens if token in model1.wv)
#     return vector_sum1 / len(tokens)

# # Apply Word Embedding to lemmatized titles
# df_id['word_embedding_title'] = df_id['lemmatized_title'].apply(get_sentence_vector)
# df_id['word_embedding_abstract'] = df_id['lemmatized_abstract'].apply(get_sentence_vector)


In [ ]:
#df_id['word_embedding_abstract']

In [ ]:
#df_id['word_embedding_title']

In [ ]:
#df_id.to_csv("chunck100k.csv")

In [ ]:
df =df_id.copy()

In [ ]:
# import pandas as pd
# from transformers import AutoTokenizer, AutoModel
# import torch
# from scipy.spatial.distance import cosine
# import chromadb
# from chromadb.config import Settings
# from bert_score import score as bert_score
# import random

# # Load the dataset
# # data = {
# #     "_id": ["65a756c9a99c69a267b682b1", "65a756c9a99c69a267b682b2", "65a756c9a99c69a267b682b3", "65a756c9a99c69a267b682b4", "65a756c9a99c69a267b682b5"],
# #     "title": [
# #         "whither expert social affordances and cultural knowledge bases question answer systems",
# #         "solving the imprecise weight coefficients knapsack problem",
# #         "combination of general antithetic transformations and methods for reducing the variance in the content",
# #         "clustering worstcase execution times for componentbased systems classical techniques",
# #         "reverse engineering goal models from legacy code"
# #     ],
# #     "abstract": [
# #         ['commun', 'base', 'question', 'answer', 'system'],
# #         ['paper', 'investig', 'solv', 'imprecis', 'weight'],
# #         ['sever', 'method', 'reduc', 'varianc', 'content'],
# #         ['componenbas', 'system', 'classic', 'techniqu'],
# #         ['revers', 'engin', 'process', 'aim', 'reconstruct']
# #     ],
# # }


# Load the SciBERT model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

# Get embeddings for titles and abstracts
df['title_embeddings'] = df['title'].apply(lambda x: get_embeddings(x).squeeze().numpy())
df['abstract_embeddings'] = df['abstract'].apply(lambda x: get_embeddings(' '.join(x)).squeeze().numpy())

# Convert numpy arrays to lists
df['title_embeddings'] = df['title_embeddings'].apply(lambda x: x.tolist())
df['abstract_embeddings'] = df['abstract_embeddings'].apply(lambda x: x.tolist())

# Convert lists in 'abstract' to strings for metadata
df['abstract_str'] = df['abstract'].apply(lambda x: ' '.join(x))

# Connect to Chroma and create collections
client = chromadb.Client(Settings())
title_collection = client.create_collection("title_embeddings")
abstract_collection = client.create_collection("abstract_embeddings")

# Add embeddings to collections
ids = df["_id"].tolist()
title_embeddings = df["title_embeddings"].tolist()
abstract_embeddings = df["abstract_embeddings"].tolist()
title_metadatas = df[["title"]].to_dict(orient="records")
abstract_metadatas = df[["abstract_str"]].to_dict(orient="records")

title_collection.add(ids=ids, embeddings=title_embeddings, metadatas=title_metadatas)
abstract_collection.add(ids=ids, embeddings=abstract_embeddings, metadatas=abstract_metadatas)

# Function to compute similarity
def compute_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)

# Function to find similar texts
def find_similar_texts(user_input, embedding_column, top_n=5):
    user_embedding = get_embeddings(user_input).squeeze().numpy().tolist()
    
    similarities = []
    for idx, row in df.iterrows():
        sim = compute_similarity(user_embedding, row[embedding_column])
        similarities.append((row['_id'], sim))
    
    # Sort by similarity and get top N results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Function for re-ranking the results
def re_rank_results(similar_texts):
    # Your re-ranking logic here
    return similar_texts  # For simplicity, returning the same list

# Updated function to find similar texts and re-rank them
def find_and_rerank_similar_texts(user_input, embedding_column, top_n=5):
    similar_texts = find_similar_texts(user_input, embedding_column, top_n)
    re_ranked_texts = re_rank_results(similar_texts)
    return re_ranked_texts

# Function to recommend papers (example random recommendation)
def recommend_papers(user_input, num_recommendations=5):
    recommended_papers = random.sample(df["_id"].tolist(), num_recommendations)
    return recommended_papers

# Function to calculate BERTScore for recommended papers
def calculate_bertscore_for_recommendations(user_input, recommended_papers):
    candidates = [user_input] * len(recommended_papers)
    references = [df[df["_id"] == paper_id]["abstract_str"].values[0] for paper_id in recommended_papers]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1_score": F1.mean().item()
    }

# Example usage
user_input = "recommendation system based on semantic analysis"
similar_titles = find_and_rerank_similar_texts(user_input, 'title_embeddings')
similar_abstracts = find_and_rerank_similar_texts(user_input, 'abstract_embeddings')
recommended_papers = recommend_papers(user_input)

# Display the results for titles
print("Similar Titles:")
for _id, similarity in similar_titles:
    title = df[df["_id"] == _id]["title"].values[0]
    print(f"Title: {title}, Similarity: {similarity}")

# Display the results for abstracts
print("Similar Abstracts:")
for _id, similarity in similar_abstracts:
    title = df[df["_id"] == _id]["title"].values[0]
    print(f"Title: {title}, Similarity: {similarity}")

# Display recommended papers
print("Recommended Papers:")
for paper_id in recommended_papers:
    title = df[df["_id"] == paper_id]["title"].values[0]
    print(f"Paper ID: {paper_id}, Title: {title}")

# Calculate BERTScore for recommended papers
bert_metrics_recommended_papers = calculate_bertscore_for_recommendations(user_input, recommended_papers)

# Display the BERTScore for recommended papers
print("BERTScore Metrics for Recommended Papers:")
print(f"Precision: {bert_metrics_recommended_papers['precision']:.2f}")
print(f"Recall: {bert_metrics_recommended_papers['recall']:.2f}")
print(f"F1 Score: {bert_metrics_recommended_papers['f1_score']:.2f}")


In [ ]:
# import pandas as pd
# from transformers import AutoTokenizer, AutoModel
# import torch
# from scipy.spatial.distance import cosine
# import chromadb
# from chromadb.config import Settings
# from bert_score import score as bert_score
# import matplotlib.pyplot as plt

# # # Load the dataset
# # data = {
# #     "_id": ["65a756c9a99c69a267b682b1", "65a756c9a99c69a267b682b2", "65a756c9a99c69a267b682b3", "65a756c9a99c69a267b682b4", "65a756c9a99c69a267b682b5"],
# #     "title": [
# #         "whither expert social affordances and cultural knowledge bases question answer systems",
# #         "solving the imprecise weight coefficients knapsack problem",
# #         "combination of general antithetic transformations and methods for reducing the variance in the content",
# #         "clustering worstcase execution times for componentbased systems classical techniques",
# #         "reverse engineering goal models from legacy code"
# #     ],
# #     "abstract": [
# #         ['commun', 'base', 'question', 'answer', 'system'],
# #         ['paper', 'investig', 'solv', 'imprecis', 'weight'],
# #         ['sever', 'method', 'reduc', 'varianc', 'content'],
# #         ['componenbas', 'system', 'classic', 'techniqu'],
# #         ['revers', 'engin', 'process', 'aim', 'reconstruct']
# #     ],
# # }

# # df = pd.DataFrame(data)
# df=pd.read_csv('chunck1.csv')
# # Load the SciBERT model and tokenizer
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Function to get embeddings
# def get_embeddings(text):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
#     with torch.no_grad():
#         embeddings = model(**inputs).last_hidden_state.mean(dim=1)
#     return embeddings

# # Get embeddings for titles and abstracts
# df['title_embeddings'] = df['title'].apply(lambda x: get_embeddings(x).squeeze().numpy())
# df['abstract_embeddings'] = df['abstract'].apply(lambda x: get_embeddings(' '.join(x)).squeeze().numpy())

# # Convert numpy arrays to lists
# df['title_embeddings'] = df['title_embeddings'].apply(lambda x: x.tolist())
# df['abstract_embeddings'] = df['abstract_embeddings'].apply(lambda x: x.tolist())

# # Convert lists in 'abstract' to strings for metadata
# df['abstract_str'] = df['abstract'].apply(lambda x: ' '.join(x))

# # Ensure IDs are strings
# df['_id'] = df['_id'].astype(str)

# # Connect to Chroma and create collections
# client = chromadb.Client(Settings())
# title_collection = client.create_collection("title_embeddings")
# abstract_collection = client.create_collection("abstract_embeddings")

# # Add embeddings to collections
# ids = df["_id"].tolist()
# title_embeddings = df["title_embeddings"].tolist()
# abstract_embeddings = df["abstract_embeddings"].tolist()
# title_metadatas = df[["title"]].to_dict(orient="records")
# abstract_metadatas = df[["abstract_str"]].to_dict(orient="records")

# title_collection.add(ids=ids, embeddings=title_embeddings, metadatas=title_metadatas)
# abstract_collection.add(ids=ids, embeddings=abstract_embeddings, metadatas=abstract_metadatas)

# # Function to compute similarity
# def compute_similarity(embedding1, embedding2):
#     return 1 - cosine(embedding1, embedding2)

# # Function to find similar texts
# def find_similar_texts(user_input, embedding_column, top_n=5):
#     user_embedding = get_embeddings(user_input).squeeze().numpy().tolist()
    
#     similarities = []
#     for idx, row in df.iterrows():
#         sim = compute_similarity(user_embedding, row[embedding_column])
#         similarities.append((row['_id'], sim))
    
#     # Sort by similarity and get top N results
#     similarities.sort(key=lambda x: x[1], reverse=True)
#     return similarities[:top_n]

# # Function for re-ranking the results
# def re_rank_results(similar_texts):
#     # Your re-ranking logic here
#     return similar_texts  # For simplicity, returning the same list

# # Updated function to find similar texts and re-rank them
# def find_and_rerank_similar_texts(user_input, embedding_column, top_n=5):
#     similar_texts = find_similar_texts(user_input, embedding_column, top_n)
#     re_ranked_texts = re_rank_results(similar_texts)
#     return re_ranked_texts

# # Function to recommend papers (example random recommendation)
# def recommend_papers(user_input, num_recommendations=5):
#     recommended_papers = df["_id"].tolist()[:num_recommendations]  # Simplified for demonstration
#     return recommended_papers

# # Function to calculate BERTScore for recommended papers
# def calculate_bertscore_for_recommendations(user_input, recommended_papers):
#     candidates = [user_input] * len(recommended_papers)
#     references = [df[df["_id"] == paper_id]["abstract_str"].values[0] for paper_id in recommended_papers]
#     P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
#     return {
#         "precision": P.mean().item(),
#         "recall": R.mean().item(),
#         "f1_score": F1.mean().item()
#     }

# # Example usage
# user_input = "recommendation system based on semantic analysis"
# similar_titles = find_and_rerank_similar_texts(user_input, 'title_embeddings')
# similar_abstracts = find_and_rerank_similar_texts(user_input, 'abstract_embeddings')
# recommended_papers = recommend_papers(user_input)

# # Display the results for titles
# print("Similar Titles:")
# for _id, similarity in similar_titles:
#     title = df[df["_id"] == _id]["title"].values[0]
#     print(f"Title: {title}, Similarity: {similarity}")

# # Display the results for abstracts
# print("Similar Abstracts:")
# for _id, similarity in similar_abstracts:
#     title = df[df["_id"] == _id]["title"].values[0]
#     print(f"Title: {title}, Similarity: {similarity}")

# # Display recommended papers
# print("Recommended Papers:")
# for paper_id in recommended_papers:
#     title = df[df["_id"] == paper_id]["title"].values[0]
#     print(f"Paper ID: {paper_id}, Title: {title}")

# # Calculate BERTScore for recommended papers
# bert_metrics_recommended_papers = calculate_bertscore_for_recommendations(user_input, recommended_papers)

# # Display the BERTScore for recommended papers
# print("BERTScore Metrics for Recommended Papers:")
# print(f"Precision: {bert_metrics_recommended_papers['precision']:.2f}")
# print(f"Recall: {bert_metrics_recommended_papers['recall']:.2f}")
# print(f"F1 Score: {bert_metrics_recommended_papers['f1_score']:.2f}")

# # Plotting BERTScore output and performance metrics
# labels = ['Precision', 'Recall', 'F1 Score']
# scores = [bert_metrics_recommended_papers['precision'], bert_metrics_recommended_papers['recall'], bert_metrics_recommended_papers['f1_score']]

# plt.figure(figsize=(10, 6))
# plt.bar(labels, scores, color=['blue', 'orange', 'green'])
# plt.title('BERTScore Output and Performance Metrics')
# plt.xlabel('Metrics')
# plt.ylabel('Score')
# plt.ylim(0, 1)  # Set y-axis limit from 0 to 1
# plt.show()
